# PROYECTO 1 ALGEBRA LINEAL PARA CIENCIA DE DATOS (IMT 2210)
#### INTEGRANTES : 
#### - Benjamin Prieto
#### - Benjamin Varela
#### - Victor Bórquez

#### Primero importamos los modulos necesarios:

In [69]:
import vec

### A) Para cada usuario, crear su vector de gustos, utilizando la clase Vec. Donde el dominio son el id de las peliculas y la funcion movie.id-> rating (utilizando diccionario). Guardar los vectores en un diccionario users

Abrimos el archivo movies.csv para sacar los ids de todas las películas

In [70]:
with open('movies.csv', 'r') as docu:
    arc = docu.readlines()
    # el len(arc) es uno mas que el ult id de pelicula
    lista_pelis = []
    diccionario_pelis = {}
    for i in range(1, len(arc)):
        lista_pelis.append(i)
        id_pel, nombre = arc[i].strip('\n').split(',',1)
        id_pel = int(id_pel)
        diccionario_pelis[id_pel] = nombre
    dominio = set(lista_pelis)

Ahora que ya tenemos el dominio para crear una instancia de Vec, debemos crear un diccionario que tenga el {id de la pelicula : rating}
Para eso, abrimos el archivo ratings.csv y creamos los vectores y por cada usuario lo incorporamos a un diccionario llamado users.

In [71]:
with open('ratings.csv', 'r') as file:
    a = file.readlines()

id_actual = 1
dicc = {}
users = {} 

for linea in a:
    linea = linea.strip('\n')
    id_us, id_pel, ratio = linea.split(',', 2)
    if id_us.isnumeric() == True:
        id_us = int(id_us)
        id_pel = int(id_pel)
        ratio = int(ratio)
        if id_us == id_actual:
            dicc[id_pel] = ratio
        else:
            vector = vec.Vec(dominio, dicc)
            users[id_actual] = vector
            dicc = {}
            dicc[id_pel] = ratio
            id_actual = id_us
vector = vec.Vec(dominio, dicc)
users[id_us] = vector


### B) Ahora para cada pelicula crear su vector con los ratings que todos los usuarios le dieron a cada pelicula

Primero abrimos el archivo ratings.csv para poder crear un set de una lista con todos los ids de usuarios(corresponde al dominio del vector de una pelicula)

In [72]:
with open('ratings.csv', 'r') as file:
    a = file.readlines()

id_actual = 1
lista_ids_us = []
for linea in a:
    linea = linea.strip('\n')
    id_us, id_pel, ratio = linea.split(',', 2)
    if id_us.isnumeric() == True:
        id_us = int(id_us)
        if id_us == id_actual:
            if id_us not in lista_ids_us:
                lista_ids_us.append(id_us)
        else:
            id_actual = id_us
            lista_ids_us.append(id_us)
dominio2 = set(lista_ids_us)

Como el archivo de ratings.csv estaba ordenado a partir de los ids de los usuarios, es decir, si la linea 16 era una calificacion del user 1, significa que todas las filas anteriores tambien eran ratings del user 1. Se nos ocurrio efectuar el mismo orden pero en relacion con la segunda columna(id_pelicula)

Entonces ahora debemos crear un diccionario que tenga {id_usuario : rating} y luego agregar el vector a un diccionario llamado movies

In [73]:
def ordenar_segundo(lista):
    return(lista[1])

lista_ordenada_pelis = []
for linea in a:
    linea = linea.strip('\n')
    id_us, id_pel, ratio = linea.split(',', 2)
    if id_us.isnumeric() == True:
        lista_ordenada_pelis.append([int(id_us), int(id_pel), int(ratio)])

lista_ordenada_pelis.sort(key = ordenar_segundo)

pel_actual = 1
dicc = {}
movies = {}
for linea in lista_ordenada_pelis:
    id_us = linea[0]
    id_pel = linea[1]
    ratio = linea[2]
    
    if id_pel == pel_actual:
        dicc[id_us] = ratio
    else:
        vector = vec.Vec(dominio2, dicc)
        movies[pel_actual] = vector
        dicc = {}
        dicc[id_us] = ratio
        pel_actual = id_pel
vector = vec.Vec(dominio2, dicc)
movies[id_pel] = vector
    

### C) Crear una función que entregue los k usuarios del diccionario users mas similares al usuario dado

In [74]:
def sim(u, v):
    punto =  vec.dot(u,v)
    punto_2 = (vec.dot(u,u))**(1/2)
    punto_3 = (vec.dot(v, v))**(1/2)
    r = punto/(punto_2*punto_3)
    return r

def vecinos(users, user_id, k):
    usuario = users[user_id]
    lista_sim = []
    for i in users:
        if i != user_id:
            simi = sim(usuario, users[i])
            lista_sim.append([i, simi])
    lista_sim.sort(key = ordenar_segundo, reverse = True )
    lista_vecinos = []
    for i in range(k):
        lista_vecinos.append(tuple(lista_sim[i]))
    return lista_vecinos


### D) Predecir el rating de un usuario a una pelicula a partir de sus usuarios similares

    -ACTIVIDAD 1: Si tomamos 50 vecinos cercanos del user 100, cuantos han calificado la pelicula 286

In [75]:
var = vecinos(users, 100, 50)
id_pelicula = 286
lista_vecinos_correctos = []
for a in var:
    id_usuario = a[0]
    if id_pelicula in users[id_usuario].f:
        lista_vecinos_correctos.append(id_usuario)

cant_vecinos = len(lista_vecinos_correctos)
f'Dentro de los 50 vecinos cercanos del usuario 100, {cant_vecinos} han calificado la pelicula 286'

'Dentro de los 50 vecinos cercanos del usuario 100, 42 han calificado la pelicula 286'

    -ACTIVIDAD 2: Porque esta cantidad de vecinos no es la adecuada?

No es razonable usar esta cantidad de vecinos porque si nos ponemos a analizar, el vecino mas lejano(dentro de este rango de 50 y que ademas cumple la condicion) ni siquiera tiene un 40% de similitud respecto al usuario 100. Entonces el rating promedio del usuario 100 para esta pelicula no seria tan representativo.

Por ejemplo, si llevamos esta situación a un caso de la vida real. Si uno quiere ir a ver una pelicula pero no esta seguro si le ira a gustar lo que hace es preguntar a gente cercana si es que la ha visto. Uno no le va a preguntar a 50 personas por una recomendación.

    -ACTIVIDAD 3: Cuantos vecinos habria que considerar para obtener un rating razonable?

Nosotros creemos que un k = 10 seria acorde porque entrega una muestra mejor representativa de los vecinos del user 100

In [76]:
var = vecinos(users, 100, 10)
id_pelicula = 286
lista_vecinos_correctos = []
suma_ptjs = 0
for a in var:
    id_usuario = a[0]
    if id_pelicula in users[id_usuario].f:
        suma_ptjs += users[id_usuario].f[id_pelicula]
        lista_vecinos_correctos.append(id_usuario)
promedio = suma_ptjs / len(lista_vecinos_correctos)
display(f'Dentro de los 10 vecinos cercanos del usuario 100, {len(lista_vecinos_correctos)} han calificado la pelicula 286')

'Dentro de los 10 vecinos cercanos del usuario 100, 8 han calificado la pelicula 286'

Ya que de los 10 usuarios con gustos mas similares, un 80% de los usuarios han calificado la pelicula. Esto nos deberia entregar una muestra mas representativa del promedio

    -ACTIVIDAD 4: Calcular el promedio para el k propuesto

In [77]:
f'A partir de los datos de los 10 usuarios mas cercanos al usuario 100, el rating de este user a la pelicula 286 seria de {promedio} pts.'

'A partir de los datos de los 10 usuarios mas cercanos al usuario 100, el rating de este user a la pelicula 286 seria de 3.75 pts.'

### E) Ver el perfil de los usuarios cercanos al user 400 y encontrar los 30 vecinos de dicho usuario y calcula el vector promedio como una combinacion convexa de los 30 vectores con todos los coeficientes iguales a 1/30. Luego apartir del vector promedio, imprimir el nombre de las 10  películas con mayor rating en orden decreciente

In [78]:
cercanos = vecinos(users, 400, 30)

puerta = False
for i in cercanos:
    user_actual = i[0]
    if puerta == False:
        suma = vec.scalar_mul(users[user_actual],1/30)
        puerta = True
    else:
        actual = vec.scalar_mul(users[user_actual],1/30)
        suma = vec.add(suma, actual)

decreciente = sorted(suma.f.items(), key = lambda x:x[1], reverse = True)


for a in range(10):
    id_peli = decreciente[a][0]
    print(diccionario_pelis[id_peli])

Titanic (1997)
Air Force One (1997)
Contact (1997)
Scream (1996)
"Saint, The (1997)"
Conspiracy Theory (1997)
"Full Monty, The (1997)"
Liar Liar (1997)
Scream 2 (1997)
"English Patient, The (1996)"


F)¿Que simboliza aplicar la ecuacion a los vectores de usuarios construidos en el item a?

Nos entrega que tan parecidos son usuarios entre si, ya que retorna un numero entre 0 y 1, que mientras mas cerca este del 1 significa que mas similares son ambos usuarios.